In [1]:
# Libraries

import xarray as xr
import pandas as pd


In [53]:
# File path
# temperature in kelvin
temp_path = "data_files/P_CMI5_RCMO_CM20_TA-_0002m_Euro_NUT0_S197001010000_E209812312100_INS_TIM_01m_NA-_cdf_org_01_RCP85_NA---_NA---.csv"

# Skip metadata rows and read the data
temp_df = pd.read_csv(
    temp_path, 
    skiprows=52,  # Skip rows up to the actual data header
    sep=',',      # Specify the correct delimiter (comma in this case)
    engine='python',  # Use Python engine for flexible parsing
    on_bad_lines='skip'  # Skip any problematic rows
)

# temp_df.info()
# Display the first few rows to verify
# print(temp_df.head())

# Ensure 'Date' is parsed as a datetime column
temp_df['Date'] = pd.to_datetime(temp_df['Date'])

# Filter rows between 2024 and 2098
temp_pred = temp_df[temp_df['Date'].dt.year >= 2024]

# Select only 'Date' and 'UK' columns
temp_uk = temp_pred[['Date', 'UK']]

# Display the resulting data
print(temp_uk)



           Date          UK
648  2024-01-31  278.318442
649  2024-02-29  279.679320
650  2024-03-31  281.119786
651  2024-04-30  281.463725
652  2024-05-31  283.513798
...         ...         ...
1543 2098-08-31  292.090237
1544 2098-09-30  290.784606
1545 2098-10-31  288.767088
1546 2098-11-30  281.167716
1547 2098-12-31  281.603563

[900 rows x 2 columns]


In [ ]:
# File path
# precipitation in meters
# https://confluence.ecmwf.int/display/CKB/Climate+and+energy+indicators+for+Europe+Variable+Fact+Sheet%3A+Precipitation
precip_path = "data_files/P_CMI5_RCMO_CM20_TP-_0000m_Euro_NUT0_S197001010000_E209812312100_ACC_TIM_01m_NA-_cdf_org_01_RCP85_NA---_NA---.csv"

# Skip metadata rows and read the data
precip_df = pd.read_csv(
    precip_path, 
    skiprows=52,  # Skip rows up to the actual data header
    sep=',',      # Specify the correct delimiter (comma in this case)
    engine='python',  # Use Python engine for flexible parsing
    on_bad_lines='skip'  # Skip any problematic rows
)

# Display the first few rows to verify
# print(precip_df.head())

# Ensure 'Date' is parsed as a datetime column
precip_df['Date'] = pd.to_datetime(precip_df['Date'])

# Filter rows between 2024 and 2098
precip_pred = precip_df[precip_df['Date'].dt.year >= 2024]

# Select only 'Date' and 'UK' columns
precip_uk = precip_pred[['Date', 'UK']]

# Display the resulting data
print(precip_uk)



           Date        UK
648  2024-01-31  0.112557
649  2024-02-29  0.111266
650  2024-03-31  0.084483
651  2024-04-30  0.071342
652  2024-05-31  0.073365
...         ...       ...
1543 2098-08-31  0.062422
1544 2098-09-30  0.080680
1545 2098-10-31  0.085424
1546 2098-11-30  0.048578
1547 2098-12-31  0.097036

[900 rows x 2 columns]


In [52]:
# File path
# wind speed Metres per second (m/s or m s-1) 10 meters
# https://confluence.ecmwf.int/display/CKB/Climate+and+energy+indicators+for+Europe+Variable+Fact+Sheet%3A+Wind+Speed
wind_path = "data_files/P_CMI5_RCMO_CM20_WS-_0010m_Euro_NUT0_S197001010000_E209812312100_INS_TIM_01m_NA-_cdf_org_01_RCP85_NA---_NA---.csv"

# Skip metadata rows and read the data
wind_df = pd.read_csv(
    wind_path, 
    skiprows=52,  # Skip rows up to the actual data header
    sep=',',      # Specify the correct delimiter (comma in this case)
    engine='python',  # Use Python engine for flexible parsing
    on_bad_lines='skip'  # Skip any problematic rows
)

# Display the first few rows to verify
# print(wind_df.head())

# Ensure 'Date' is parsed as a datetime column
wind_df['Date'] = pd.to_datetime(wind_df['Date'])

# Filter rows between 2024 and 2098
wind_pred = wind_df[wind_df['Date'].dt.year >= 2024]

# Select only 'Date' and 'UK' columns
wind_uk = wind_pred[['Date', 'UK']]

# Display the resulting data
print(wind_uk)


           Date        UK
648  2024-01-31  5.663002
649  2024-02-29  6.870473
650  2024-03-31  5.629517
651  2024-04-30  3.811083
652  2024-05-31  4.063906
...         ...       ...
1543 2098-08-31  3.462024
1544 2098-09-30  4.471978
1545 2098-10-31  4.152824
1546 2098-11-30  3.629667
1547 2098-12-31  4.886961

[900 rows x 2 columns]
